<a href="https://colab.research.google.com/github/simulai/TopoWMA/blob/main/MinimumViableDao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install torch torchvision  # 如果需要升级或安装其他依赖时执行


MVD 模块：

In [39]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision
import torchvision.transforms as transforms

# 修改后的 MVD 模型：增加非线性变换、残差融合和可学习融合权重alpha
class MVD(nn.Module):
    def __init__(self, input_dim):
        super(MVD, self).__init__()
        self.tao_gate = nn.Parameter(torch.tensor(0.618))  # 黄金分割初始值
        self.fc = nn.Linear(input_dim, 10)  # CIFAR-10 有 10 类

        # 对原始输入进行非线性变换（残差分支）
        self.residual_transform = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.ReLU()
        )
        # 可学习的融合权重，控制两部分融合比例
        self.alpha = nn.Parameter(torch.tensor(0.5))

    def forward(self, x):
        norm = x.norm(p=2, dim=1, keepdim=True)
        modulation = 1 + self.tao_gate * torch.sin(norm)
        modulated = x * modulation

        # 残差分支：对原始输入进行额外非线性变换
        residual = self.residual_transform(x)

        # 加权融合：让模型自动平衡两部分信息
        fused = self.alpha * modulated + (1 - self.alpha) * residual

        return self.fc(fused)

# Baseline 模型（仅包含纯线性层）
class Baseline(nn.Module):
    def __init__(self, input_dim):
        super(Baseline, self).__init__()
        self.fc = nn.Linear(input_dim, 10)

    def forward(self, x):
        return self.fc(x)


In [40]:

# 数据预处理：使用 CIFAR-10 数据集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

batch_size = 64
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 设备设置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_dim = 3 * 32 * 32  # 展平后的维度

# 实例化模型并迁移到设备
model_mvd = MVD(input_dim).to(device)
model_baseline = Baseline(input_dim).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer_mvd = optim.Adam(model_mvd.parameters(), lr=0.001)
optimizer_baseline = optim.Adam(model_baseline.parameters(), lr=0.001)

# 采用 Cosine Annealing 学习率调度器，训练周期延长至 100 个 epoch
num_epochs = 100
scheduler_mvd = lr_scheduler.CosineAnnealingLR(optimizer_mvd, T_max=num_epochs)
scheduler_baseline = lr_scheduler.CosineAnnealingLR(optimizer_baseline, T_max=num_epochs)




Files already downloaded and verified
Files already downloaded and verified


**模型实例化及训练设置**

In [41]:

# 训练函数
def train(model, optimizer, loader, device):
    model.train()
    total_loss = 0
    for images, labels in loader:
        images = images.view(images.size(0), -1).to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# 评估函数
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.view(images.size(0), -1).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total



In [42]:

# 主训练循环
print("Training MVD model with modified residual fusion and learning rate scheduling...")
for epoch in range(num_epochs):
    loss_mvd = train(model_mvd, optimizer_mvd, train_loader, device)
    acc_mvd = evaluate(model_mvd, test_loader, device)

    loss_baseline = train(model_baseline, optimizer_baseline, train_loader, device)
    acc_baseline = evaluate(model_baseline, test_loader, device)

    scheduler_mvd.step()
    scheduler_baseline.step()

    print(f"Epoch {epoch+1}:")
    print(f" MVD      -> Loss: {loss_mvd:.4f}, Accuracy: {acc_mvd*100:.2f}%")
    print(f" Baseline -> Loss: {loss_baseline:.4f}, Accuracy: {acc_baseline*100:.2f}%\n")

Training MVD model with modified residual fusion and learning rate scheduling...
Epoch 1:
 MVD      -> Loss: 1.6922, Accuracy: 46.55%
 Baseline -> Loss: 1.8611, Accuracy: 35.92%

Epoch 2:
 MVD      -> Loss: 1.4887, Accuracy: 48.49%
 Baseline -> Loss: 1.7937, Accuracy: 38.29%

Epoch 3:
 MVD      -> Loss: 1.3930, Accuracy: 46.35%
 Baseline -> Loss: 1.7719, Accuracy: 36.93%

Epoch 4:
 MVD      -> Loss: 1.3243, Accuracy: 50.15%
 Baseline -> Loss: 1.7554, Accuracy: 36.46%

Epoch 5:
 MVD      -> Loss: 1.2473, Accuracy: 52.25%
 Baseline -> Loss: 1.7509, Accuracy: 37.83%

Epoch 6:
 MVD      -> Loss: 1.1804, Accuracy: 51.22%
 Baseline -> Loss: 1.7444, Accuracy: 37.17%

Epoch 7:
 MVD      -> Loss: 1.1115, Accuracy: 51.70%
 Baseline -> Loss: 1.7398, Accuracy: 38.56%

Epoch 8:
 MVD      -> Loss: 1.0492, Accuracy: 50.95%
 Baseline -> Loss: 1.7359, Accuracy: 37.55%

Epoch 9:
 MVD      -> Loss: 0.9788, Accuracy: 51.38%
 Baseline -> Loss: 1.7292, Accuracy: 37.66%

Epoch 10:
 MVD      -> Loss: 0.9093, 

In [43]:
# 在实例化模型后添加
print("\nMVD Structure:")
print(model_mvd)
print("\nBaseline Structure:")
print(model_baseline)

# 计算参数量
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nMVD Parameters: {count_parameters(model_mvd):,}")
print(f"Baseline Parameters: {count_parameters(model_baseline):,}")


MVD Structure:
MVD(
  (fc): Linear(in_features=3072, out_features=10, bias=True)
  (residual_transform): Sequential(
    (0): Linear(in_features=3072, out_features=3072, bias=True)
    (1): ReLU()
  )
)

Baseline Structure:
Baseline(
  (fc): Linear(in_features=3072, out_features=10, bias=True)
)

MVD Parameters: 9,470,988
Baseline Parameters: 30,730
